In [1]:
import json
import requests
import pandas as pd 
from datetime import datetime

In [2]:
api_key = "OiPvDJBdnu3RPHoQq9LQ7QHfnQjd1pxx"

latitude = 10.7758439
longitude = 106.7017555
hourly = "1h"
units="metric"

url = f"https://api.tomorrow.io/v4/weather/forecast?location=HoChiMinh&timesteps={hourly}&units={units}&apikey={api_key}"
response = requests.get(url)

data = response.json()
json_str = json.dumps(data, indent=4)
print(json_str)

{
    "timelines": {
        "hourly": [
            {
                "time": "2025-07-19T04:00:00Z",
                "values": {
                    "altimeterSetting": 1004.87,
                    "cloudBase": 0.6,
                    "cloudCeiling": 1.7,
                    "cloudCover": 100,
                    "dewPoint": 25.2,
                    "evapotranspiration": 0.185,
                    "freezingRainIntensity": 0,
                    "humidity": 77,
                    "iceAccumulation": 0,
                    "iceAccumulationLwe": 0,
                    "precipitationProbability": 35,
                    "pressureSeaLevel": 1005.08,
                    "pressureSurfaceLevel": 1002.97,
                    "rainAccumulation": 0.7,
                    "rainIntensity": 6.2,
                    "sleetAccumulation": 0,
                    "sleetAccumulationLwe": 0,
                    "sleetIntensity": 0,
                    "snowAccumulation": 0,
                    "snowAcc

In [4]:
if response.status_code == 200:
    data = response.json() #dict
    weather_data = data.get("timelines", {}).get("hourly", [])

    flat_data = [] # vì là JSON lồng, nên cần làm phẳng vd: time temp hum ...
    for entry in weather_data:
        record = {"time": entry["time"]} # lấy ra time
        record.update(entry.get("values", {})) # lấy ra các giá trị trong values
        flat_data.append(record)

    df = pd.DataFrame(flat_data)

    # Lấy các cột cần thiết
    important_cols = [
        "time", "temperature", "temperatureApparent", "dewPoint", "humidity", 
        "windSpeed", "windDirection", "windGust", "pressureSurfaceLevel",
        "cloudCover","precipitationProbability", "uvIndex", "ezHeatStressIndex"
    ]

    final_cols = [col for col in important_cols if col in df.columns]
    df = df[final_cols]

    df.to_csv("weather_data.csv", index=False)
    print("Đã lưu dữ liệu thô vào stagging weather_data.csv")
else:
    print(f" Lỗi API - status code: {response.status_code}")

Đã lưu dữ liệu thô vào stagging weather_data.csv


In [3]:
df = pd.read_csv("weather_data.csv")
df.head()

,time,temperature,temperatureApparent,dewPoint,humidity,windSpeed,windDirection,windGust,pressureSurfaceLevel,cloudCover,precipitationProbability,uvIndex
0,2025-07-18T16:00:00Z,28.6,35.1,24.7,88,2.7,12,3.2,1004.48,100,0,0.0
1,2025-07-18T17:00:00Z,25.7,25.7,25.3,98,0.2,322,0.5,1006.42,100,0,0.0
2,2025-07-18T18:00:00Z,25.6,25.6,25.2,98,0.8,224,1.4,1005.84,100,0,0.0
3,2025-07-18T19:00:00Z,25.8,25.8,24.7,94,1.7,237,3.5,1005.13,100,0,0.0
4,2025-07-18T20:00:00Z,25.6,25.6,24.1,91,2.3,248,4.6,1004.53,99,0,0.0
